In [1]:
from main import model, labelers
import pandas as pd

In [2]:
import torch

In [24]:
sd = torch.load('./dashing-dawn-197-e26.bin')

In [25]:
model.load_state_dict(sd)
entemb = model.encoder.entity_embeddings.weight.detach().cpu().numpy()
print(entemb.shape)
id2cid = labelers["id_labeler"].encoder.classes_
idorder = pd.DataFrame({"CMTE_ID": id2cid})

def read_frame(header_file, data_file, dtypes={}):
    header = pd.read_csv(header_file)
    dt = {c: str for c in header.columns}
    dt.update(dtypes)
    data = pd.read_csv(data_file, sep="|", names=header.columns, dtype=dt)
    return data

def read_cm(year, basedir="./data"):
    cm = read_frame(
        f"{basedir}/cm_header_file.csv",
        f"{basedir}/{year}/cm.txt",
        dtypes={
            c: "str"
            for c in (
                "CMTE_DSGN",
                "CMTE_TP",
                "CMTE_PTY_AFFILIATION",
                "CMTE_FILING_FREQ",
            )
        },
    )
    return cm

cmdf = (
    idorder.join(
        pd.concat([read_cm(2020), read_cm(2022), read_cm(2024)])
        .drop_duplicates(subset=["CMTE_ID"], keep="last")
        .set_index("CMTE_ID"),
        on="CMTE_ID",
    )
    .dropna(subset=["CMTE_NM"])
    .fillna("N/A")
)
namedemb = entemb[cmdf.index]

(18629, 256)


In [26]:
from umap import UMAP
import umap.plot as upl

In [32]:
uop = UMAP(verbose=True, n_jobs=-1, metric='cosine')

In [33]:
p2d = uop.fit_transform(namedemb)

UMAP( verbose=True)
Fri Nov 24 20:53:48 2023 Construct fuzzy simplicial set
Fri Nov 24 20:53:48 2023 Finding Nearest Neighbors
Fri Nov 24 20:53:48 2023 Building RP forest with 11 trees
Fri Nov 24 20:53:48 2023 NN descent for 14 iterations
	 1  /  14
	 2  /  14
	 3  /  14
	 4  /  14
	 5  /  14
	 6  /  14
	 7  /  14
	 8  /  14
	 9  /  14
	Stopping threshold met -- exiting after 9 iterations
Fri Nov 24 20:54:01 2023 Finished Nearest Neighbor Search
Fri Nov 24 20:54:02 2023 Construct embedding


Epochs completed:   0%|            0/200 [00:00]

	completed  0  /  200 epochs
	completed  20  /  200 epochs
	completed  40  /  200 epochs
	completed  60  /  200 epochs
	completed  80  /  200 epochs
	completed  100  /  200 epochs
	completed  120  /  200 epochs
	completed  140  /  200 epochs
	completed  160  /  200 epochs
	completed  180  /  200 epochs
Fri Nov 24 20:54:14 2023 Finished embedding


In [34]:
upl.output_notebook()

Loading BokehJS ...

In [35]:
p = upl.interactive(uop, hover_data=cmdf, point_size=2, labels=cmdf['CMTE_DSGN'].values)
upl.show(p)
